In [129]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_20220101-20251231.csv')

#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print(num_days)



[4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [130]:
#grab rates
rater = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\rates_definition.csv')

#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'Pretreat Tanks'
#consumable = 'Air'

#grabs the usage rate of consumable and sums it up
sum_usage = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list())
#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#prints
print("Consumable used = ", sum_usage)
print("Consumable generated = ", sum_generated)
net_consumable = sum_generated - sum_usage
print("net consumable per day= ", net_consumable)



Consumable used =  0.005555556
Consumable generated =  0
net consumable per day=  -0.005555556


In [131]:
import math

raw_to_send = [i * rate for i in num_days]
consumable_to_send = []
current_sum = 0

rate = -(net_consumable)
#if the rate of depletion is less than 1 unit per day then we send just 1 unit on certain shipments
if rate < 1 and rate > 0:
    for i, value in enumerate(raw_to_send):
        current_sum+= value
        if current_sum <= 1:
            consumable_to_send.append(0)
        else:
            consumable_to_send.append(1)
            current_sum = 0
    print(consumable_to_send)
else:
    #otherwise we take the raw and round up
    consumable_to_send = [math.ceil(i * rate) for i in num_days]
    print(consumable_to_send)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]


In [132]:
##RUN ALL or this wont work
dock_days_list.pop()

readable = pd.DataFrame(
    {'Docking Days': dock_days_list,
     'Amount of Consumable to Send': consumable_to_send
    })

display(readable)

Docking Days  Amount of Consumable to Send
0     2/17/2022                             0
1     2/21/2022                             0
2     3/18/2022                             0
3      4/9/2022                             0
4     4/27/2022                             0
..          ...                           ...
57     9/3/2025                             0
58    9/10/2025                             0
59    10/5/2025                             0
60    11/2/2025                             0
61   11/27/2025                             1

[62 rows x 2 columns]